In [1]:
import requests
import pandas as pd
import numpy as np
import time
from bs4 import BeautifulSoup

In [2]:
def parse_avito(url, delay):
    response = requests.get(url)
    tree = BeautifulSoup(response.content, 'html.parser')
    try:
        max_pages = int(tree.find('div', {'class' : 'js-pages pagination-pagination-2j5na'}).find_all(
            'span', {'class' : 'pagination-item-1WyVp'})[-2].text)
    except:
        max_pages = 1
    names = [] 
    prices = []
    metro  = []
    distance = []
    date = []
    links = []
    for page in range(1, max_pages + 1):
        time.sleep(delay)
        response = requests.get(url + '&p=' + str(page))
        tree = BeautifulSoup(response.content, 'html.parser')
        flats = tree.find_all('div', {'class' : 'description item_table-description'})
        for flat in flats:
            names.append(flat.h3.a.get('title'))
            prices.append(flat.find('span', {'class': 'snippet-price'}).text.strip().replace('\n',''))
            try:
                metro.append(flat.find('span', {'class': 'item-address-georeferences-item__content'}).text.strip())
            except:
                metro.append(None)
            try:
                distance.append(flat.find('span', {'class': 'item-address-georeferences-item__after'}).text.strip())
            except:
                distance.append(None)
            date.append(flat.find('div', {'class': 'snippet-date-info'})['data-tooltip'])
            links.append('avito.ru' + flat.h3.a.get('href'))
    table = pd.DataFrame({'Название': names, 'Цена': prices, 'Метро': metro, 'Расстояние до метро': distance, 'Дата': date, 'Ссылка': links})
    return table

In [3]:
max_price = str(65000)
rooms = str(3)
delay = 0.8
url_part1 = 'https://www.avito.ru/moskva/kvartiry/sdam/na_dlitelnyy_srok/' + rooms + '-komnatnye-ASgBAQICAkSSA8gQ8AeQUgFAzAgUklk?cd=1&pmax='
url_part2 = '&f=ASgBAQICAkSSA8gQ8AeQUgJAzAgUklmsvg0UpMc1'
url_avito = url_part1 + max_price + '&proprofile=1&metro=' + url_part2

In [4]:
df_avito = parse_avito(url_avito, delay)
df_avito

,Название,Цена,Метро,Расстояние до метро,Дата,Ссылка
0,"3-к квартира, 75 м², 3/14 эт. в Москве",52 000 ₽ в месяц,Лермонтовский проспект,700 м,29 сентября 13:02,avito.ru/moskva/kvartiry/3-k_kvartira_75_m_314...
1,"3-к квартира, 56 м², 9/9 эт. в Москве",41 000 ₽ в месяц,Выхино,700 м,30 сентября 18:31,avito.ru/moskva/kvartiry/3-k_kvartira_56_m_99_...
2,"3-к квартира, 80 м², 10/17 эт. в Москве",50 000 ₽ в месяц,Улица Скобелевская,"2,3 км",30 сентября 14:27,avito.ru/moskva/kvartiry/3-k_kvartira_80_m_101...
3,"3-к квартира, 56 м², 5/5 эт. в Москве",50 000 ₽ в месяц,Войковская,"1,1 км",30 сентября 14:13,avito.ru/moskva/kvartiry/3-k_kvartira_56_m_55_...
4,"3-к квартира, 71 м², 12/13 эт. в Москве",48 000 ₽ в месяц,Шипиловская,800 м,30 сентября 11:16,avito.ru/moskva/kvartiry/3-k_kvartira_71_m_121...
...,...,...,...,...,...,...
743,"3-к квартира, 62 м², 3/12 эт. в Москве",60 000 ₽ в месяц,Люблино,800 м,3 сентября 10:18,avito.ru/moskva/kvartiry/3-k_kvartira_62_m_312...
744,"3-к квартира, 56.6 м², 2/5 эт. в Москве",45 000 ₽ в месяц,Улица Дмитриевского,"1,1 км",5 сентября 13:01,avito.ru/moskva/kvartiry/3-k_kvartira_56.6_m_2...
745,"3-к квартира, 83 м², 7/12 эт. в Зеленограде",60 000 ₽ в месяц,Зеленоград,None,8 сентября 10:39,avito.ru/moskva_zelenograd/kvartiry/3-k_kvarti...
746,"3-к квартира, 55 м², 2/5 эт. в Москве",50 000 ₽ в месяц,Тушинская,600 м,3 сентября 21:02,avito.ru/moskva/kvartiry/3-k_kvartira_55_m_25_...
